### 도로표지판과 신호등 클래스를 담고 있는 이미지 파일만 선별

우리는 2번 클래스(도로표지판)과 3번 클래스(신호등)을 감지하는 모델을 만들 것

따라서 2번 클래스와 3번 클래스 라벨이 없는 이미지는 사용할 필요가 없으므로 제외

단, 제외된 이미지들 중 일부는 모델 정확도를 높이고 오탐지(False Positive)를 줄이기 위한 Background image로 추후에 사용할 것임

In [ ]:
import os
import json
import pandas as pd
from tqdm import tqdm

In [ ]:
img_path = "E:/pgu_traffic_dataset/cleaned/images"
ann_path = "E:/pgu_traffic_dataset/cleaned/annotations"
ann_file_list = os.listdir(ann_path)

In [ ]:
#2번 혹은 3번 클래스를 담고 있는 파일만 걸러내 selected_ann_file_list에 파일명을 담아 놓는다.
#반복문을 도는 김에 2번 클래스의 개수와 3번 클래스의 개수를 계산해 균형이 맞는지 확인한다.
selected_ann_file_list = []
class_2_cnt = 0
class_3_cnt = 0
for ann_file in tqdm(ann_file_list):
    ann_str = open("{}/{}".format(ann_path, ann_file), "r").read() #가장 첫번째 파일을 불러와 내용 확인
    ann = json.loads(ann_str)
    this_class_2_cnt = len([x for x in ann["annotations"] if x["category_id"] == 2])
    this_class_3_cnt = len([x for x in ann["annotations"] if x["category_id"] == 3])
    if this_class_2_cnt > 0 or this_class_3_cnt > 0:
        selected_ann_file_list.append(ann_file)

    class_2_cnt += this_class_2_cnt
    class_3_cnt += this_class_3_cnt

print("2, 3번 클래스가 포함된 이미지 : {}개".format(len(selected_ann_file_list))) # -> 30802개 중 28547개의 이미지가 선택됨. 2255개의 이미지는 사용하지 않는 것으로 함
print("2번 클래스의 개수 : {}개".format(class_2_cnt)) # -> 280773개
print("3번 클래스의 개수 : {}개".format(class_3_cnt)) # -> 138825개

클래스 별 개수는 균형이 맞아야 모델 성능이 좋아진다. 지금처럼 개수 차이가 나는 상황에서는 3개의 선택지가 있다.

1. 3번 클래스가 포함되어 있는 이미지를 더 추가한다.

2. 2번 클래스가 포함되어 있는 이미지를 삭제한다.

3. 데이터 불균형이 있지만 그냥 진행한다.

1번이 모델 성능에 가장 좋겠으나, 추가할 이미지가 없고 데이터 양이 충분하므로 2번 방식으로 진행한다.

In [ ]:
#2번 클래스가 포함된 이미지를 그냥 제거하면 3번 클래스의 개수도 줄어들게 된다.
#따라서, 3번 클래스(신호등)보다 2번 클래스(표지판)가 2배 이상 많이 포함된 이미지일 경우에만 제거하는 것을 기준으로 하여 진행한다.
ann_file_list_to_remove = []
class_2_cnt = 0
class_3_cnt = 0
for ann_file in tqdm(selected_ann_file_list):
    ann_str = open("{}/{}".format(ann_path, ann_file), "r").read() #가장 첫번째 파일을 불러와 내용 확인
    ann = json.loads(ann_str)
    this_class_2_cnt = len([x for x in ann["annotations"] if x["category_id"] == 2])
    this_class_3_cnt = len([x for x in ann["annotations"] if x["category_id"] == 3])
    if this_class_3_cnt*2 <= this_class_2_cnt:
        ann_file_list_to_remove.append(ann_file)
    else:
        class_2_cnt += this_class_2_cnt
        class_3_cnt += this_class_3_cnt
print("제거할 이미지 수 : {}".format(len(ann_file_list_to_remove))) # -> 17000개
print("남은 이미지 수 : {}".format(len(selected_ann_file_list) - len(ann_file_list_to_remove))) # -> 11547개
print("2번 클래스의 개수 : {}".format(class_2_cnt)) # -> 117619개
print("3번 클래스의 개수 : {}".format(class_3_cnt)) # -> 101977개

In [ ]:
#최종 파일 목록을 만들고 파일로 저장해 놓는다.
final_selected_ann_file_list = [x for x in selected_ann_file_list if x not in ann_file_list_to_remove]
final_selected_ann_file_json = json.dumps(final_selected_ann_file_list)
open("선별된 최종 파일 목록.json", "w", encoding="utf-8").write(final_selected_ann_file_json)
len(final_selected_ann_file_list)